In [3]:
import numpy as np
import os
from typing import Dict, Tuple
import re

import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras

import keras.layers as l
from keras import models, callbacks, utils, losses

In [26]:
text = ''
with open('garri-potter-i-metody-racionalnogo-myshleniya.txt', 'r', encoding='utf_8_sig') as file:
    text = file.read(300000)

def get_features_target(seq: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 32

words = list(filter(None, [re.sub('[^а-яА-ЯёЁ0-9 ,-]', '', s).strip() for s in text.split('.')]))
alphabet = np.array(sorted(set(' '.join(words).split(' '))))

word_index = {char: i for i, char in enumerate(alphabet)}
index_word = {i: char for i, char in enumerate(alphabet)}

sequences = Dataset.from_tensor_slices(np.array([word_index[word] for word in ' '.join(words).split()])).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

In [27]:
model = keras.Sequential([
    l.Embedding(len(alphabet), BATCH_SIZE, batch_input_shape=[BATCH_SIZE, None]),
    l.SimpleRNN(128, return_sequences=True, stateful=True),
    l.Dense(len(alphabet) / 2, activation='relu'),
    l.Dense(len(alphabet))
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=25, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/25
42/42 [==============================] - 91s 2s/step - loss: 8.5707 - accuracy: 0.0206
Epoch 2/25
42/42 [==============================] - 98s 2s/step - loss: 7.9596 - accuracy: 0.0216
Epoch 3/25
42/42 [==============================] - 101s 2s/step - loss: 7.9543 - accuracy: 0.0236
Epoch 4/25
42/42 [==============================] - 99s 2s/step - loss: 7.7105 - accuracy: 0.0255
Epoch 5/25
42/42 [==============================] - 90s 2s/step - loss: 7.4641 - accuracy: 0.0274
Epoch 6/25
42/42 [==============================] - 90s 2s/step - loss: 7.2015 - accuracy: 0.0315
Epoch 7/25
42/42 [==============================] - 86s 2s/step - loss: 6.8558 - accuracy: 0.0359
Epoch 8/25
42/42 [==============================] - 85s 2s/step - loss: 6.2695 - accuracy: 0.0432
Epoch 9/25
42/42 [==============================] - 86s 2s/step - loss: 5.2541 - accuracy: 0.1199
Epoch 10/25
42/42 [==============================] - 86s 2s/step - loss: 4.4041 - accuracy: 0.2417
Epoch 11/25
42/42 

In [30]:
def predict(sample, model, tokenizer, vocabulary, n_next, temperature, batch_size, word = False):
    if word:
        sample_vector = [tokenizer[word] for word in sample.split()]
    else:
        sample_vector = [tokenizer[char] for char in sample]
    predicted = sample_vector
    sample_tensor = tf.expand_dims(sample_vector, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    for i in range(n_next):
        pred = model(sample_tensor)
        pred = pred[0].numpy() / temperature
        pred = tf.random.categorical(pred, num_samples=1)[-1, 0].numpy()
        predicted.append(pred)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([pred], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    pred_seq = [vocabulary[i] for i in predicted]
    generated = ' '.join(pred_seq) if word else ''.join(pred_seq)
    return generated

In [37]:
print(predict(
    sample='Гарри',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.5,
    batch_size=BATCH_SIZE,
    word=True
))

Гарри выкрикнул её один и ещё несколько капель мистер Поттер протянул всё Если будто на руке или ещё за самом деле


In [46]:
print(predict(
    sample='Гермиона',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.25,
    batch_size=BATCH_SIZE,
    word=True
))

Гермиона из них Гермиона мальчик мешочек с золотом есть все глаза Но он снова того, что Гарри Поттер он только как


In [55]:
print(predict(
    sample='Он',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=30,
    temperature=0.8,
    batch_size=BATCH_SIZE,
    word=True
))

Он на одну И он только Я она этот её с ваших наконец их Гермиона Ладно, не мог этот мысли, но я тебя об этом что застыло до собой из всех


In [56]:
print(predict(
    sample='Вдруг',
    model=model,
    tokenizer=word_index,
    vocabulary=index_word,
    n_next=20,
    temperature=0.4,
    batch_size=BATCH_SIZE,
    word=True
))

Вдруг Гарри Поттер ещё ли Поттер он будет их мальчик форме Драко одно Гарри Поттер ещё это только ещё ещё просто
